<h1>Battle of the Neighborhoods - Clustering the Neighborhoods of Paris</h1>
<h2>1. Introduction</h2>
<p>TripAdvisor, the travel planning and booking site, unveiled the 2018 Travelers' Choice ™ Award-winning destinations. This ranking recognizes the favorite destinations of travelers around the world. In 2018, Paris took 1st place in the “World” ranking, ahead of London, Rome and New York.</p>
<p>Travelers' Choice ™ Awards winners are determined based on an algorithm that assesses the quantity and quality of reviews of hotels, restaurants and attractions in destinations around the world, gathered over a 12-month period, as well as the booking interest of travelers when they search the web.</p>


<h2>2. Business Problem</h2>
<p>The objective of this project is to provide tourists heading to Paris the relevant information about the best places to stay in the city while also taking into consideration the best attractions. We will try to find the most interesting neighborhood to stay in using an Airbnb service.</p>

<h2>3. Data</h2>
<p>We will be using the following datasets provided by Airbnb:</p>
<ul>
    <li><a href="http://data.insideairbnb.com/france/ile-de-france/paris/2020-12-14/visualisations/neighbourhoods.csv">http://data.insideairbnb.com/france/ile-de-france/paris/2020-12-14/visualisations/neighbourhoods.csv</a></li>
    <li><a href="http://data.insideairbnb.com/france/ile-de-france/paris/2020-12-14/visualisations/neighbourhoods.geojson">http://data.insideairbnb.com/france/ile-de-france/paris/2020-12-14/visualisations/neighbourhoods.geojson</a></li>

<p>We will also be leveraging the Foursquare API in order to discover information about different venues belonging to a neighborhood and a borough. Foursquare is a location data provider that will provide us with the relevant data involving our venues.</p>

In [1]:
!wget -q -O 'neighborhoods.geojson' http://data.insideairbnb.com/france/ile-de-france/paris/2020-12-14/visualisations/neighbourhoods.geojson